## Customer Churn Project

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

### Importing PySpark

In [1]:
import findspark
findspark.init('/home/ubuntu/spark-3.1.1-bin-hadoop3.2')

In [2]:
from pyspark.sql import SparkSession

In [3]:
#Initialized Spark Session
spark = SparkSession.builder.appName('logproject').getOrCreate()

### Importing the Data

In [4]:
df = spark.read.csv('customer_churn.csv',inferSchema=True,header=True)

## Exploratory Data Analysis

In [5]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [6]:
df.describe().show(5)

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|               null|                null|                null|0.16666666666666666|
| stddev| 

In [7]:
df.head(1)

[Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date='2013-08-30 07:00:40', Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1)]

In [8]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

### Missing Values

In [9]:
from pyspark.sql.functions import isnan, when, count, col

In [10]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+
|Names|Age|Total_Purchase|Account_Manager|Years|Num_Sites|Onboard_date|Location|Company|Churn|
+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+
|    0|  0|             0|              0|    0|        0|           0|       0|      0|    0|
+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+



### Shape of the dataset

In [11]:
print((df.count(),len(df.columns)))

(900, 10)


## Data Preprocessing

In [9]:
from pyspark.ml.feature import VectorAssembler

In [10]:
assembler = VectorAssembler(inputCols=['Age','Total_Purchase','Account_Manager','Years','Num_Sites'],
                           outputCol='features')

In [11]:
output = assembler.transform(df)

In [12]:
output.show(1)

+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------+-----+--------------------+
|           Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|   Company|Churn|            features|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------+-----+--------------------+
|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|Harvey LLC|    1|[42.0,11066.8,0.0...|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------+-----+--------------------+
only showing top 1 row



In [13]:
final_data = output.select(['features','Churn'])

In [14]:
final_data.show(1)

+--------------------+-----+
|            features|Churn|
+--------------------+-----+
|[42.0,11066.8,0.0...|    1|
+--------------------+-----+
only showing top 1 row



## Model Building

### Train Test Split

In [15]:
train_data , test_data = final_data.randomSplit([0.7,0.3],seed=101)

### Logistic Regression

In [16]:
from pyspark.ml.classification import LogisticRegression

In [17]:
#Creating Logistic Regression Instance
log_reg = LogisticRegression(labelCol='Churn')

In [18]:
#Fitting the model to the data
log_reg_model = log_reg.fit(train_data)

#### Training Data Summary

In [19]:
training_summary = log_reg_model.summary

In [20]:
training_summary.predictions.describe().show()

+-------+------------------+------------------+
|summary|             Churn|        prediction|
+-------+------------------+------------------+
|  count|               628|               628|
|   mean|0.1735668789808917|0.1321656050955414|
| stddev|0.3790385095109955|0.3389406867257665|
|    min|               0.0|               0.0|
|    max|               1.0|               1.0|
+-------+------------------+------------------+



### Evaluation

In [21]:
test_results = log_reg_model.evaluate(test_data)

In [22]:
test_results.predictions.show(5)

+--------------------+-----+--------------------+--------------------+----------+
|            features|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[27.0,8628.8,1.0,...|    0|[5.71606738466376...|[0.99671817172168...|       0.0|
|[28.0,8670.98,0.0...|    0|[8.17623235378818...|[0.99971881945157...|       0.0|
|[29.0,10203.18,1....|    0|[3.94315325691176...|[0.98098172099756...|       0.0|
|[29.0,11274.46,1....|    0|[4.68306058794832...|[0.99083413224459...|       0.0|
|[29.0,13255.05,1....|    0|[4.31365120788893...|[0.98679219076935...|       0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 5 rows



#### Binary Classification Evaluator

In [23]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [24]:
test_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Churn')

#### AUC Score

In [25]:
auc = test_eval.evaluate(test_results.predictions)
auc

0.7466476612818077

#### Accuracy

In [26]:
test_results.accuracy

0.8933823529411765

## Predicting on New Data

In [29]:
#fitting the model to the whole data
final_logreg_model = log_reg.fit(final_data)

In [30]:
new_customers = spark.read.csv('new_customers.csv',inferSchema=True,header=True)

In [32]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [33]:
#Transforming through assembler first
test_new_customers = assembler.transform(new_customers)

In [35]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [36]:
#Predictions
final_results = final_logreg_model.transform(test_new_customers)

In [37]:
final_results.show(1)

+-------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------+--------------------+--------------------+--------------------+----------+
|        Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location| Company|            features|       rawPrediction|         probability|prediction|
+-------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------+--------------------+--------------------+--------------------+----------+
|Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|King Ltd|[37.0,9935.53,1.0...|[2.22168705251407...|[0.90218018099701...|       0.0|
+-------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------+--------------------+--------------------+--------------------+----------+
only showing top 1 row



In [40]:
final_results.select(['Company','prediction']).show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



Ok! That is it! Now we know that we should assign Acocunt Managers to Cannon-Benson,Barron-Robertson,Sexton-Golden, and Parks-Robbins!